In [1]:
from requests import get
import pandas as pd


In [2]:
all_countries=get('https://raw.githubusercontent.com/everypolitician/everypolitician-data/master/countries.json')
j=all_countries.json()
len(j)

233

In [4]:
aus_hr_url=j[13]['legislatures'][0]['popolo_url']
aus_hr_data=get(aus_hr_url)
aus_hr_df=pd.json_normalize(aus_hr_data.json()['persons'])

In [5]:
aus_hr_df['birth_date']=pd.to_datetime(aus_hr_df['birth_date'])
aus_hr_df.head()

,birth_date,contact_details,family_name,gender,given_name,id,identifiers,links,name,sort_name,death_date,other_names,email,image,images,honorific_prefix
0,1954-03-21,"[{'type': 'twitter', 'value': 'nueday'}]",Kelly,female,De-Anne,004e5075-34b7-4ba5-b53b-f992a28ea6dc,"[{'identifier': '10357', 'scheme': 'everypolit...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",De-Anne Kelly,"Kelly, De-Anne",NaN,NaN,NaN,NaN,NaN,NaN
1,1925-06-15,NaN,Millar,male,Percival,00c2ed39-73db-43aa-a5b7-bea4508dbd5e,"[{'identifier': '10457', 'scheme': 'everypolit...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",Percival Millar,"Millar, Percival",2017-11-28,"[{'lang': 'en', 'name': 'Clarrie Millar', 'not...",NaN,NaN,NaN,NaN
2,1951-07-04,"[{'type': 'email', 'value': 'john.alexander.mp...",Alexander,male,John,010ba615-63c1-466b-a425-5d127e12b8c3,"[{'identifier': 'M3M', 'scheme': 'aph'}, {'ide...","[{'note': 'Wikimedia Commons', 'url': 'https:/...",John Alexander,"Alexander, John",NaN,"[{'lang': 'ar', 'name': 'جون ألكسندر', 'note':...",john.alexander.mp@aph.gov.au,http://www.aph.gov.au//api/parliamentarian/M3M...,[{'url': 'http://www.aph.gov.au//api/parliamen...,NaN
3,1954-02-13,"[{'type': 'twitter', 'value': 'senronno'}]",Ronaldson,male,Michael,012988dd-b93f-4bfe-95b8-27f3e5a56ee7,"[{'identifier': 'xt4', 'scheme': 'aph'}, {'ide...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",Michael Ronaldson,"Ronaldson, Michael",NaN,"[{'lang': 'ja', 'name': 'マイケル・ロナルドソン', 'note':...",NaN,NaN,NaN,NaN
4,1946-08-28,NaN,Andren,male,Peter,0267b78a-440a-414f-bf35-5284b9ebaa0d,"[{'identifier': '10012', 'scheme': 'everypolit...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",Peter Andren,"Andren, Peter",2007-11-03,NaN,NaN,NaN,NaN,NaN


In [14]:
aus_hr_df.shape

(515, 16)

In [6]:
aus_hr_df['birth_date']=pd.to_datetime(aus_hr_df['birth_date'])
monthly_df=aus_hr_df.groupby(aus_hr_df['birth_date'].dt.month).count()[['name']]
monthly_df.index=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
monthly_df['percentage']=(monthly_df['name']/monthly_df['name'].sum()*100).round(1).astype(str) + '%'
monthly_df.rename(columns={'name':'count'},inplace=True)
monthly_df.transpose()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
count,45,38,52,44,40,46,47,41,43,43,38,38
percentage,8.7%,7.4%,10.1%,8.5%,7.8%,8.9%,9.1%,8.0%,8.3%,8.3%,7.4%,7.4%


In [8]:
for country in j:
    for legislature in country['legislatures']:
        

In [11]:
# Functions to get country and legislature wise data

def get_legislature_data(link):
    data=get(link)
    df=pd.json_normalize(data.json()['persons'])
    return df

def get_country_data(country):
    return pd.concat([get_legislature_data(legislature['popolo_url']) for legislature in country['legislatures']])

In [13]:
# Create a dataframe with data for all countries
df=pd.concat([get_country_data(country) for country in j])
df

,gender,id,identifiers,image,images,name,birth_date,links,other_names,given_name,contact_details,family_name,death_date,email,sort_name,honorific_prefix,honorific_suffix,national_identity,summary,patronymic_name
0,female,0a93b26d-ebc5-44f5-b4fa-935ae209620f,"[{'identifier': '127', 'scheme': 'everypolitic...",http://www.parlamentra.org/upload/iblock/d1a/g...,[{'url': 'http://www.parlamentra.org/upload/ib...,Гамисония Эмма Алексеевна,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,male,0ac7e64e-b723-4bdb-85f6-81ff217a70fa,"[{'identifier': '211', 'scheme': 'everypolitic...",http://www.parlamentra.org/upload/iblock/4dd/I...,[{'url': 'http://www.parlamentra.org/upload/ib...,Барганджия Гурам Юрьевич,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,male,0b515281-445d-49dd-9044-d886d85f0970,"[{'identifier': '157', 'scheme': 'everypolitic...",http://www.parlamentra.org/upload/iblock/244/c...,[{'url': 'http://www.parlamentra.org/upload/ib...,Чамагуа Леонид Михайлович,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,male,12d1a33c-daa4-496c-965a-4cd4749eda78,"[{'identifier': '130', 'scheme': 'everypolitic...",http://www.parlamentra.org/upload/iblock/c3b/t...,[{'url': 'http://www.parlamentra.org/upload/ib...,Цвижба Отари Шотович,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,male,15a55ab3-c81c-4f5f-ae75-ea5446dd5ef6,"[{'identifier': '138', 'scheme': 'everypolitic...",http://www.parlamentra.org/upload/iblock/30b/y...,[{'url': 'http://www.parlamentra.org/upload/ib...,Язычба Заур Гайдарович,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,male,ed4838aa-19f8-493c-b8c4-58780b5b0d84,"[{'identifier': '112', 'scheme': 'everypolitic...",http://www.lagtinget.ax/files/sjolund_folke.jpg,[{'url': 'http://www.lagtinget.ax/files/sjolun...,Sjölund Folke,1943-12-16,"[{'note': 'Wikipedia (fi)', 'url': 'https://fi...","[{'lang': 'en', 'name': 'Folke Sjölund', 'note...",Folke,NaN,Sjölund,2013-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,male,f105a4a0-5fef-4be7-ab73-9376923491a7,NaN,https://www.lagtinget.ax/sites/www.lagtinget.a...,[{'url': 'https://www.lagtinget.ax/sites/www.l...,John Holmberg,1967,NaN,NaN,NaN,"[{'type': 'email', 'value': 'john.holmberg@lag...",NaN,NaN,john.holmberg@lagtinget.ax,NaN,NaN,NaN,NaN,NaN,NaN
57,male,f4f995ae-126c-468f-8f26-deec1e26adc2,"[{'identifier': '372', 'scheme': 'everypolitic...",http://www.lagtinget.ax/files/asumaa_tony.jpg,[{'url': 'http://www.lagtinget.ax/files/asumaa...,Asumaa Tony,1968-09-15,"[{'note': 'Wikipedia (en)', 'url': 'https://en...","[{'lang': 'en', 'name': 'Tony Asumaa', 'note':...",Tony,NaN,Asumaa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,male,fe40bb08-5025-473d-af52-bf1e30ecc5e6,"[{'identifier': '88', 'scheme': 'everypolitici...",http://www.lagtinget.ax/files/sundblom_torsten...,[{'url': 'http://www.lagtinget.ax/files/sundbl...,Sundblom Torsten,1951-09-15,"[{'note': 'Wikipedia (fi)', 'url': 'https://fi...","[{'lang': 'en', 'name': 'Torsten Sundblom', 'n...",Torsten,"[{'type': 'email', 'value': 'torsten.sundblom@...",NaN,NaN,torsten.sundblom@lagtinget.ax,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.shape

(78382, 20)

In [18]:
df.to_csv('data.csv',index=True)